In [1]:
import tensorflow as tf

# VGG网络

In [2]:
# 定义VGG⽹络中的卷积块：卷积层的个数，卷积层中卷积核的个数
def vgg_block(num_convs, num_filters):
 # 构建序列模型
 blk = tf.keras.models.Sequential()
 # 遍历所有的卷积层
 for _ in range(num_convs):
    # 每个卷积层：num_filter个卷积核，卷积核⼤⼩为3*3，paddin
    blk.add(tf.keras.layers.Conv2D(num_filters,kernel_size=3,padding='same',activation='relu')),
 # 卷积块最后是⼀个最⼤池化，窗⼝⼤⼩为2*2，步⻓为2
 blk.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2)),
 return blk

In [3]:
# 定义VGG⽹络
def vgg(conv_arch):
    # 构建序列模型
    net = tf.keras.models.Sequential()
    # 根据conv_arch⽣成卷积部分
    for (num_convs, num_filters) in conv_arch:
        net.add(vgg_block(num_convs, num_filters))
    # 卷积块序列后添加全连接层
    net.add(tf.keras.models.Sequential([
        # 将特征图展成⼀维向量
        tf.keras.layers.Flatten(),
        # 全连接层：4096个神经元，激活函数是relu
        tf.keras.layers.Dense(4096, activation='relu'),
        # 随机失活
        tf.keras.layers.Dropout(0.5),
        # 全连接层：4096个神经元，激活函数是relu
        tf.keras.layers.Dense(4096, activation='relu'),
        # 随机失活
        tf.keras.layers.Dropout(0.5),
        # 全连接层：10个神经元，激活函数是softmax
        tf.keras.layers.Dense(10, activation='softmax')
    ]))

    return net

In [4]:
# 定义5个卷积块，指明每个卷积块中的卷积层个数及相应的卷积核个数
conv_arch = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))

In [5]:
# ⽹络实例化
net = vgg(conv_arch)

In [6]:
# 构造输⼊X，并将其送⼊到net⽹络中
X = tf.random.uniform((1,224,224,1))
y = net(X)
# 通过net.summay()查看⽹络的形状
net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (1, 112, 112, 64)         37568     
_________________________________________________________________
sequential_2 (Sequential)    (1, 56, 56, 128)          221440    
_________________________________________________________________
sequential_3 (Sequential)    (1, 28, 28, 256)          1475328   
_________________________________________________________________
sequential_4 (Sequential)    (1, 14, 14, 512)          5899776   
_________________________________________________________________
sequential_5 (Sequential)    (1, 7, 7, 512)            7079424   
_________________________________________________________________
sequential_6 (Sequential)    (1, 10)                   119586826 
Total params: 134,300,362
Trainable params: 134,300,362
Non-trainable params: 0
__________________________________________

# 手写数字识别

In [10]:
import numpy as np
# 数据集
from tensorflow.keras.datasets import mnist

# 获取⼿写数字数据集
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# 训练集数据维度的调整：N H W C
train_images = np.reshape(train_images,(train_images.shape[0],train_images.shape[1],train_images.shape[2],1))
# 测试集数据维度的调整：N H W C
test_images = np.reshape(test_images,(test_images.shape[0],test_images.shape[1],test_images.shape[2],1))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
# 定义两个⽅法随机抽取部分样本演示
# 获取训练集数据
def get_train(size):
 # 随机⽣成要抽样的样本的索引
 index = np.random.randint(0, np.shape(train_images)[0],size)
 # 将这些数据resize成22*227⼤⼩
 resized_images = tf.image.resize_with_pad(train_images[index],224,224,)
 # 返回抽取的
 return resized_images.numpy(), train_labels[index]

# 获取测试集数据
def get_test(size):
 # 随机⽣成要抽样的样本的索引
 index = np.random.randint(0, np.shape(test_images)[0], size)
 # 将这些数据resize成224*224⼤⼩
 resized_images = tf.image.resize_with_pad(test_images[index],224,224,)
 # 返回抽样的测试样本
 return resized_images.numpy(), test_labels[index]

In [12]:
# 获取训练样本和测试样本
train_images,train_labels = get_train(256)
test_images,test_labels = get_test(128)

In [13]:
# 模型编译
# 指定优化器，损失函数和评价指标
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0)
net.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [14]:
# 模型训练：指定训练数据，batchsize,epoch,验证集
net.fit(train_images,train_labels,batch_size=128,epochs=3,verbose=1,validation_split=0.1)

Epoch 1/3


2024-04-23 06:31:49.711038: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng15{k5=1,k6=0,k7=1,k10=1} for conv (f32[128,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[128,1,224,224]{3,2,1,0}, f32[64,1,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-04-23 06:31:51.560317: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.10754, expected 0.564662
2024-04-23 06:31:51.560419: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.59741, expected 1.05453
2024-04-23 06:31:51.560439: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 1.73904, expected 1.19615
2024-04-23 06:31:51.560452: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137

1/2 ━━━━━━━━━━━━━━━━━━━━ 2:12 133s/step - accuracy: 0.0781 - loss: 2.5912

2024-04-23 06:34:00.832407: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng15{k5=1,k6=0,k7=1,k10=1} for conv (f32[102,64,224,224]{3,2,1,0}, u8[0]{0}) custom-call(f32[102,1,224,224]{3,2,1,0}, f32[64,1,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-04-23 06:34:02.098438: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.35547, expected 1.04633
2024-04-23 06:34:02.098503: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.65821, expected 1.34907
2024-04-23 06:34:02.098521: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 1.80334, expected 1.4942
2024-04-23 06:34:02.098538: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] 

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101s/step - accuracy: 0.0825 - loss: 2.5560  

2024-04-23 06:35:39.743470: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 0.84189, expected 0.195114
2024-04-23 06:35:39.743534: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 1.65739, expected 1.01062
2024-04-23 06:35:39.743549: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 1.2831, expected 0.636325
2024-04-23 06:35:39.743564: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 1.66445, expected 1.01768
2024-04-23 06:35:39.743579: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.4983, expected 0.851522
2024-04-23 06:35:39.743590: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 1.13419, expected 0.487417
2024-04-23 06:35:39.743600: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 1.82225, expected 1.17548
2024-04-23 06:35:39.743611: E external/local_xla/xla/

2/2 ━━━━━━━━━━━━━━━━━━━━ 246s 114s/step - accuracy: 0.0840 - loss: 2.5442 - val_accuracy: 0.0385 - val_loss: 2.3193
Epoch 2/3


W0000 00:00:1713854151.622651      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 693ms/step - accuracy: 0.1932 - loss: 2.2553 - val_accuracy: 0.1538 - val_loss: 2.2780
Epoch 3/3
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 692ms/step - accuracy: 0.2671 - loss: 2.1842 - val_accuracy: 0.2692 - val_loss: 2.1307


In [15]:
# 指定测试数据
net.evaluate(test_images,test_labels,verbose=1)

2024-04-23 06:36:11.016839: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 1.2936, expected 0.622453
2024-04-23 06:36:11.016904: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 2.24655, expected 1.5754
2024-04-23 06:36:11.016914: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.29063, expected 1.61949
2024-04-23 06:36:11.016921: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 1.82898, expected 1.15784
2024-04-23 06:36:11.016929: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 1.91122, expected 1.24007
2024-04-23 06:36:11.016936: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.07738, expected 1.40624
2024-04-23 06:36:11.016944: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.02675, expected 1.3556
2024-04-23 06:36:11.016952: E external/local_xla/xla/serv

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.4045 - loss: 1.9811

W0000 00:00:1713854186.381102      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4/4 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.4021 - loss: 1.9885


[1.9994637966156006, 0.3984375]